Testing data positive feedback dynamic (20%)


In [1]:
import os
import pandas as pd
import numpy as np
import random
import time
import tensorflow as tf
import math

In [2]:
def relu(x):
    return np.maximum(0,x)  

def softmax(x):
    exp_x = np.exp(x)
    softmax_x = exp_x / np.sum(exp_x)
    return softmax_x 

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [3]:
user_following = np.load('../Data/npy/user_following_1489.npy')
all_3374 = np.load('../Data/npy/all_3374D.npy')
user_category = np.load('../Data/npy/user_category_1489.npy')
YouTuber_category = np.load('../Data/npy/YouTuber_category_0.7.npy')
active_users = np.load('../Data/npy/active_userID_1489.npy')

In [5]:
#The number of followers for each YouTuber
YouTuber_followers = np.sum(user_following, axis=0)
#print(YouTuber_followers)
over5 = 0
for num in YouTuber_followers:
    if num >= 5:
        over5+=1
print('The num of followers over 5:',over5)

The num of followers over 5: 76


In [6]:
print('user_following shape ',user_following.shape)
print('all_3374 shape ',all_3374.shape)
print('user_category shape ',user_category.shape)
print('YouTuber_category shape ',YouTuber_category.shape)

user_following shape  (1489, 88)
all_3374 shape  (88, 3374)
user_category shape  (1489, 17)
YouTuber_category shape  (88, 17)


In [7]:
user_category_norm = np.zeros(user_category.shape)
for i in range(len(user_category)):
    user_category_norm[i] = user_category[i]/np.max(user_category[i])
print('user_category after normalized by max...')
print('user_category_norm shape ',user_category_norm.shape)

user_category after normalized by max...
user_category_norm shape  (1489, 17)


In [8]:
following_true = [0]*len(user_following)
for i in range(len(user_following)):
    each_user = []
    for j in range(len(user_following[i])):
        if user_following[i][j] == 1:
            each_user.append(j)
    following_true[i] = each_user
#print(following_true)

In [9]:
#最少跟最多的following 
minlen = 10000
maxlen = 0
num_of_follower = []
for i in range(len(following_true)):
    if len(following_true[i]) < minlen:
        minlen = len(following_true[i])
    if len(following_true[i]) > maxlen:
        maxlen = len(following_true[i])
    num_of_follower.append(len(following_true[i]))
print('Min number of followings ',minlen)
print('Max number of followings ',maxlen)

Min number of followings  5
Max number of followings  34


In [11]:
over_10 = 0
over_8 = 0
over_12 = 0
user_idx_over10 = []
avg_over10 = 0
for i in range(len(num_of_follower)):
    num = num_of_follower[i]
    if num >= 10:
        over_10 += 1
        user_idx_over10.append(i)
        avg_over10+=num
    if num >= 8:
        over_8 += 1
    if num >= 12:
        over_12 += 1
print('over 8:',over_8)
print('over 10:',over_10)
print('over 12:',over_12)
print('avg 10:',avg_over10/len(user_idx_over10))

over 8: 1073
over 10: 792
over 12: 499
avg 10: 13.194444444444445


In [12]:
len(user_idx_over10)

792

# Training data and Testing data

In [13]:
test_amount = 150
yt_test_amount = 20

In [14]:
user_idx = [i for i in range(len(user_following))]
#user_idx = user_idx_over10
#test_idx is the number of user for testing
test_idx = random.sample(user_idx,test_amount)

In [15]:
# Training  and Testing
train_t = [0]*(len(user_following))
train_f = [0]*(len(user_following))
# Testing 
test_t = [0]*test_amount
test_f = [0]*test_amount
test_pos = -1

for i in range(len(user_following)):
    t_for_train = []
    f_for_train = []
    if i not in test_idx: #if not in test id, just append it to true or false list
        for j in range(88):
            if user_following[i][j] == 1:
                t_for_train.append(j)
            else:
                f_for_train.append(j)
        train_t[i] = t_for_train
        train_f[i] = f_for_train
        
    else: #if in test id, choose 2 true and other 
        test_pos += 1
        temp_t = []
        temp_f = []
        for j in range(88):
            if user_following[i][j] == 1:
                temp_t.append(j)
            else:
                temp_f.append(j)
        # random choose 2 true and 8 false for test 
        if len(temp_t)*0.2 < 1:
            t_for_test = random.sample(temp_t,1)
            f_for_test  = random.sample(temp_f,yt_test_amount-1)
        else:
            if len(temp_t)*0.2 > 5:
                t_num = int(round(float(len(temp_t)*0.2)))
            else:      
                t_num = 5
            #t_num = int(round(float(len(temp_t)*0.2)))
            t_for_test = random.sample(temp_t,t_num)
            f_for_test  = random.sample(temp_f,yt_test_amount-t_num)
        test_t[test_pos] = t_for_test
        test_f[test_pos] = f_for_test
        
        #other for training
        t_for_train = [item for item in temp_t if not item in t_for_test]
        f_for_train = [item for item in temp_f if not item in f_for_test]
        train_t[i] = t_for_train
        train_f[i] = f_for_train

In [33]:
# Training  and Testing --New
train_t = [0]*(len(user_following))
train_f = [0]*(len(user_following))
# Testing 
test_t = [0]*test_amount
test_f = [0]*test_amount
test_pos = -1

for i in range(len(user_following)):
    t_for_train = []
    f_for_train = []
    if i not in test_idx: #if not in test id, just append it to true or false list
        for j in range(88):
            if user_following[i][j] == 1:
                t_for_train.append(j)
            else:
                f_for_train.append(j)
        train_t[i] = t_for_train
        train_f[i] = f_for_train
        
    else: #if in test id, choose 2 true and other 
        test_pos += 1
        temp_t = []
        temp_f = []
        for j in range(88):
            if user_following[i][j] == 1:
                temp_t.append(j)
            else:
                temp_f.append(j)
        #print(len(temp_t),int(len(temp_t)/2))
        #t_for_test = random.sample(temp_t,math.ceil(len(temp_t)/2))
        t_for_test = random.sample(temp_t,int(len(temp_t)/2))
        f_for_test  = random.sample(temp_f,yt_test_amount-len(t_for_test))
        
        test_t[test_pos] = t_for_test
        test_f[test_pos] = f_for_test
        
        #other for training
        t_for_train = [item for item in temp_t if not item in t_for_test]
        #print(len(t_for_train ))
        f_for_train = [item for item in temp_f if not item in f_for_test]
        train_t[i] = t_for_train
        train_f[i] = f_for_train

In [34]:
total_test = 0
for t in test_t:
    total_test += len(t)
avg = total_test/test_amount
avg

6.44

In [35]:
# train_t[i] 代表的是user i positive feedback
print('The length of train_t:',len(train_t))
print('The length of train_f:',len(train_t))
print('The length of test_t:',len(test_t))
print('The length of test_f:',len(test_f))

The length of train_t: 1489
The length of train_f: 1489
The length of test_t: 150
The length of test_f: 150


# Recommendation  Module

In [36]:
"""
n: the number of users
m: the number of YouTubers
k: latent dims
l: feature dims
"""
tf.reset_default_graph()
n = len(user_following)
m = 88  
k = 100
l = 3374

user = tf.placeholder(tf.int32,shape=(1,))
i = tf.placeholder(tf.int32, shape=(1,))
j = tf.placeholder(tf.int32, shape=(1,))

#多少個auxliary 
xf = tf.placeholder(tf.float32, shape=(None,l))
l_id = tf.placeholder(tf.int32, shape=(None,))
l_id_len = tf.placeholder(tf.int32,shape=(1,))
r = tf.placeholder(tf.float32,shape=(None,))


image_i = tf.placeholder(tf.float32, shape=(1,l))
image_j = tf.placeholder(tf.float32, shape=(1,l))

with tf.variable_scope("item_level"):
    user_latent = tf.get_variable("user_latent", [n, k],
                                      initializer=tf.random_normal_initializer(0,0.1,seed=3))
    item_latent = tf.get_variable("item_latent", [m, k],
                                      initializer=tf.random_normal_initializer(0,0.1,seed=3)) 
    aux_item = tf.get_variable("aux_item", [m, k],
                                      initializer=tf.random_normal_initializer(0,0.1,seed=3))
    W1 = tf.get_variable("W1", [n, k],
                                      initializer=tf.contrib.layers.xavier_initializer())
    Wu = tf.get_variable("Wu", [k,k],
                                      initializer=tf.contrib.layers.xavier_initializer())
    Wy = tf.get_variable("Wy", [m,k,k],
                                     initializer=tf.contrib.layers.xavier_initializer())
    Wa = tf.get_variable("Wa", [k,k],
                                     initializer=tf.contrib.layers.xavier_initializer())
    Wv = tf.get_variable("Wv", [k,l],
                                    initializer=tf.contrib.layers.xavier_initializer())
    
    

    aux_new = tf.get_variable("aux_new", [1,k], initializer=tf.constant_initializer(0.0))
    ########## Error part, how to get auxisize dynamically
    ####aux_size= tf.get_variable(name='aux_size', initializer=l_id.get_shape().as_list()[-1])
    
with tf.variable_scope('feature_level'):
    embedding = tf.get_variable("embedding", [150,3371],
                                     initializer=tf.contrib.layers.xavier_initializer())
    Beta = tf.get_variable("beta", [n,l],
                             # initializer=tf.contrib.layers.xavier_initializer())
                                     initializer=tf.random_normal_initializer(0.00001,0.000001,seed=10))

#lookup the latent factors by user and id
u = tf.nn.embedding_lookup(user_latent, user) #(1*k) 第幾個user latent factor
vi = tf.nn.embedding_lookup(item_latent, i) 
vj = tf.nn.embedding_lookup(item_latent, j)

w1 = tf.nn.embedding_lookup(W1, user) #(1*k)
wu = Wu
#wu = tf.squeeze(tf.nn.embedding_lookup(Wu, user)) #(k*k)
wy = tf.squeeze(tf.nn.embedding_lookup(Wy, i)) #(k*k)
wa = Wa
#wa = tf.squeeze(tf.nn.embedding_lookup(Wa, user)) #(k*k)
wv = Wv
#wv = tf.squeeze(tf.nn.embedding_lookup(Wv, user)) #(k,l)

beta = tf.nn.embedding_lookup(Beta, user) #user feature latent factor


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [41]:
#Embedding for feature vector
"""
n: the number of users
m: the number of YouTubers
k: latent dims
l: feature dims
"""
tf.reset_default_graph()
n = len(user_following)
m = 88  
k = 100
l = 3374
embedding_dims = 150

user = tf.placeholder(tf.int32,shape=(1,))
i = tf.placeholder(tf.int32, shape=(1,))
j = tf.placeholder(tf.int32, shape=(1,))

#多少個auxliary 
xf = tf.placeholder(tf.float32, shape=(None,l))
l_id = tf.placeholder(tf.int32, shape=(None,))
l_id_len = tf.placeholder(tf.int32,shape=(1,))
r = tf.placeholder(tf.float32,shape=(None,))


image_i = tf.placeholder(tf.float32, shape=(1,l))
image_j = tf.placeholder(tf.float32, shape=(1,l))

with tf.variable_scope("item_level"):
    user_latent = tf.get_variable("user_latent", [n, k],
                                      initializer=tf.random_normal_initializer(0,0.1,seed=3))
    item_latent = tf.get_variable("item_latent", [m, k],
                                      initializer=tf.random_normal_initializer(0,0.1,seed=3)) 
    aux_item = tf.get_variable("aux_item", [m, k],
                                      initializer=tf.random_normal_initializer(0,0.1,seed=3))
    W1 = tf.get_variable("W1", [n, k],
                                      initializer=tf.contrib.layers.xavier_initializer())
    Wu = tf.get_variable("Wu", [k,k],
                                      initializer=tf.contrib.layers.xavier_initializer())
    Wy = tf.get_variable("Wy", [m,k,k],
                                     initializer=tf.contrib.layers.xavier_initializer())
    Wa = tf.get_variable("Wa", [k,k],
                                     initializer=tf.contrib.layers.xavier_initializer())
    Wv = tf.get_variable("Wv", [k,l],
                                    initializer=tf.contrib.layers.xavier_initializer())
    
    aux_new = tf.get_variable("aux_new", [1,k], initializer=tf.constant_initializer(0.0))
    ########## Error part, how to get auxisize dynamically
    ####aux_size= tf.get_variable(name='aux_size', initializer=l_id.get_shape().as_list()[-1])
    
with tf.variable_scope('feature_level'):
    embedding = tf.get_variable("embedding", [embedding_dims,l],
                                     initializer=tf.contrib.layers.xavier_initializer())
    Beta = tf.get_variable("beta", [n,embedding_dims],
                             # initializer=tf.contrib.layers.xavier_initializer())
                                     initializer=tf.random_normal_initializer(0.00001,0.000001,seed=10))

#lookup the latent factors by user and id
u = tf.nn.embedding_lookup(user_latent, user) #(1*k) 第幾個user latent factor
vi = tf.nn.embedding_lookup(item_latent, i) 
vj = tf.nn.embedding_lookup(item_latent, j)

w1 = tf.nn.embedding_lookup(W1, user) #(1*k)
wu = Wu
#wu = tf.squeeze(tf.nn.embedding_lookup(Wu, user)) #(k*k)
wy = tf.squeeze(tf.nn.embedding_lookup(Wy, i)) #(k*k)
wa = Wa
#wa = tf.squeeze(tf.nn.embedding_lookup(Wa, user)) #(k*k)
wv = Wv
#wv = tf.squeeze(tf.nn.embedding_lookup(Wv, user)) #(k,l)

beta = tf.nn.embedding_lookup(Beta, user) #user feature latent factor


In [1]:
a_list=tf.Variable([])
q = tf.constant(0)
def att_cond(q,a_list):
    return tf.less(q,l_id_len[0])
def att_body(q,a_list):
    xfi = tf.expand_dims(xf[q],0) #(1,l)
    
    a_list = tf.concat([a_list,[(tf.matmul( w1, tf.nn.relu( tf.matmul(wu, u, transpose_b=True) +
        tf.matmul(wy, tf.expand_dims(tf.nn.embedding_lookup(item_latent,l_id[q]),0), transpose_b=True) +
        tf.matmul(wa, tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0), transpose_b=True) +
        tf.matmul(wv, xfi, transpose_b=True)))[0][0])*r[q]]],0)
    q += 1
    return q,  a_list

_, a_list = tf.while_loop(att_cond,att_body,[q,a_list],shape_invariants=[q.get_shape(),tf.TensorShape([None])])

a_list_soft=tf.nn.softmax(a_list)


aux_np = tf.expand_dims(tf.zeros(k),0) #dimension (1,32)
q = tf.constant(0)
def sum_att_cond(q,aux_np):
    return tf.less(q,l_id_len[0])

def sum_att_body(q,aux_np):
    #aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
    aux_np = tf.math.add_n([aux_np,a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)]) 
    q += 1
    return q, aux_np

_,aux_np = tf.while_loop(sum_att_cond,sum_att_body,[q,aux_np])

"""
for q in range(3): #取q個auxliary item
    aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
"""

aux_part = tf.matmul(aux_np, vi, transpose_b=True)
#tf.print('aux attention:',aux_np)
aux_np+=u #user_latent factor + sum (alpha*auxilary)
aux_new=tf.assign(aux_new,aux_np) #把aux_new 的 值變成aux_np


latent_i_part = tf.matmul(aux_new, vi, transpose_b=True)
feature_i_part = tf.matmul(beta,(tf.matmul(embedding,image_i, transpose_b=True)))
latent_j_part = tf.matmul(aux_new, vj, transpose_b=True)
feature_j_part = tf.matmul(beta,(tf.matmul(embedding,image_j, transpose_b=True)))
only_aux_i_part = tf.matmul(aux_np, vi, transpose_b=True)
only_aux_j_part = tf.matmul(aux_np, vj, transpose_b=True)

#矩陣中對應函數各自相乘
# ex: tf.matmul(thetav,(tf.matmul(embedding, image_i, transpose_b=True)))
xui = tf.matmul(aux_new, vi, transpose_b=True)+ tf.matmul(beta,(tf.matmul(embedding,image_i, transpose_b=True)))
xuj = tf.matmul(aux_new, vj, transpose_b=True)+ tf.matmul(beta,(tf.matmul(embedding,image_j, transpose_b=True)))

xuij = tf.subtract(xui,xuj)

norm_par = [tf.reduce_sum(tf.multiply(u, u)),tf.reduce_sum(tf.multiply(vi, vi)),tf.reduce_sum(tf.multiply(vj, vj)),
           tf.reduce_sum(tf.multiply(w1, w1)),tf.reduce_sum(tf.multiply(wu, wu)),tf.reduce_sum(tf.multiply(wy, wy)),
           tf.reduce_sum(tf.multiply(wa, wa)),tf.reduce_sum(tf.multiply(wv,wv)),tf.reduce_sum(tf.multiply(beta,beta))]
l2_norm = tf.add_n([
            0.0001 * tf.reduce_sum(tf.multiply(u, u)),
            0.0001 * tf.reduce_sum(tf.multiply(vi, vi)),
            0.0001 * tf.reduce_sum(tf.multiply(vj, vj)),
  
            0.0001 * tf.reduce_sum(tf.multiply(w1, w1)),
            0.00005 * tf.reduce_sum(tf.multiply(wu, wu)),
            0.0001 * tf.reduce_sum(tf.multiply(wy, wy)),
            0.0005 * tf.reduce_sum(tf.multiply(wa, wa)),
            0.0005 * tf.reduce_sum(tf.multiply(wv,wv)),
            
            0.001 * tf.reduce_sum(tf.multiply(beta,beta)),
            
          ])

loss = l2_norm -tf.log(tf.sigmoid(xuij)) # objective funtion
train_op = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss) #parameter optimize 
auc = tf.reduce_mean(tf.to_float(xuij > 0))

NameError: name 'tf' is not defined

In [49]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
loss_acc_list = []
t0=time.time()

#use_true=init_list_of_objects(136)
#use_test=init_list_of_objects(136)

#train_pair_t=[] #positive feedback
#train_pair_f=[] #negative feedback
train_yes_id=[] 
for q in range(5):
    print('Iteraction:',q)
    train_auc=0
    total_loss=0
    xuij_auc=0
    length = 0
    for z in range(n):
        print(z)
        """
        yes 用來存放選擇到的YouTuber feature (for auxilary)
        yesr 用來存放user對該YouTuber的喜好程度(user_category 跟 YouTuber_category的相似性)
        r_3 用來存放user 對該YouTuber種類的偏好(取max)
        """
        yes=[]
        yesr=[]
        
        
        sample=random.sample(train_t[z],len(train_t[z])) #隨機選3個sample true's YouTuber
        train_yes_id.append(sample) #sample全部丟進去
        
        #sample=random.sample(train_t[z]+train_f[z],len(train_t[z])+len(train_f[z]))
        
        #change
        r_3=np.zeros(len(sample)) 
        alpha_history = []
        a_list_history = []
        U_history = []
        Y_history = []
        
        #print(len(sample))
        #check if all YouTuber are in train_t or train_f
        #if len(train_t[z])+len(train_f[z]) != 88:
            #print(z,len(train_t[z])+len(train_f[z]))
         
        for b in range(len(sample)):
            yes.append(all_3374[sample[b]])
            yesr.append(YouTuber_category[sample[b]]*user_category_norm[z])
            #print('YouTuber_category ', YouTuber_category[sample[k]])
            #print('User_category ',user_category_norm[z])
        #print(yesr)
        
        for b in range(len(yesr)):
            r_3[b]=max(yesr[b])
        #print('r_3:',r_3)
        
        yes=np.array(yes)
        #print('user shape should be ',np.array([z]).shape)
        #print('xf shape should be ',yes.shape)
        #print('r shape should be ',np.array(r_3).shape)
        #print('l_id shape should be ',np.array(sample).shape)
        
        #not_used_list = list(set(train_t[z]).difference(set(sample)))
        
        #取positive 
        train_t_sample = random.sample(train_t[z],len(train_t[z]))
        #print('number of positive feedback', len(train_t_sample))
        
        train_f_sample = random.sample(train_f[z],20)
        for ta in train_t_sample:
            #print(ta,'--> positive feedback')
            
            pos = sample.index(ta)
            #new_sample = np.delete(sample,[pos])
            #new_yes = np.delete(yes,[pos],axis=0)
            #new_r_3 = np.delete(r_3,[pos])
            new_sample = sample
            new_yes = yes
            new_r_3 = r_3
            #print(len(yes),len(new_yes))
            #print(yes)
            #print(new_yes)
            
            
            
            #ta=random.choice(train_t[z]) #ta is true positve photo
            #train_pair_t.append(ta)
            image_1=np.expand_dims(all_3374[ta],0) #(1,2048)
            #print('Image_1 shape ',image_1.shape)
            #train_f_sample = random.sample(train_f[z],20)
            #print('True:',train_t_sample,'Now:',ta)
            #print('False:',train_f_sample)
            for b in train_f_sample:
                #print('likes:',ta,';Not likes:',b)
                #b=random.choice(train_f[z])  #b is no feedback photo
                #train_pair_f.append(b)
                image_2=np.expand_dims(all_3374[b],0) #(1,2048)
                #print('Image_2 shape',image_2.shape)
            
                #use_test[z].append(b)
                _embedding,_a_list,r3,_auc, _loss,_=sess.run([embedding,a_list,a_list_soft,auc,loss,train_op], feed_dict={user: [z],
                                        i: [ta], j: [b], xf: new_yes , l_id:new_sample, l_id_len:[len(new_sample)],r:new_r_3,
                                        image_i:image_1,image_j:image_2})
                
                #print(XUIJ)
                #print('loss=',_loss)
                #print('auc=',_auc)
                
                #print('after softmax:',r3)
                #print('before softmax:',_a_list)
                #print('embedding:',_embedding)
                #print('---------------------------------------------------')
                a_list_history.append(_a_list)
                alpha_history.append(r3)
                train_auc+=_auc
                total_loss+=_loss
                length += 1
            #now1+=1
        
        np.save('../Data/latent_factor/YRM_up10_ALL/Embedding/'+str(q)+'_'+str(z),_embedding)
    
    #print('mine:',xuij_auc/136)    
    #print('a_list_soft:',r3)
    print("total_loss:-----------------", total_loss/length)
    print("train_auc:-------------------", train_auc/length)
    print('---------------------------------------------------------------------------------------------------')
    loss_acc_list.append([total_loss/length,train_auc/length,time.time()-t0])
    print('time:',time.time()-t0,' sec')
print('Total cost ',time.time()-t0,' sec')       

Iteraction: 0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273


430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679


836
837
838
839
840
841
842
843
844
845
846
847
848
849
850
851
852
853
854
855
856
857
858
859
860
861
862
863
864
865
866
867
868
869
870
871
872
873
874
875
876
877
878
879
880
881
882
883
884
885
886
887
888
889
890
891
892
893
894
895
896
897
898
899
900
901
902
903
904
905
906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921
922
923
924
925
926
927
928
929
930
931
932
933
934
935
936
937
938
939
940
941
942
943
944
945
946
947
948
949
950
951
952
953
954
955
956
957
958
959
960
961
962
963
964
965
966
967
968
969
970
971
972
973
974
975
976
977
978
979
980
981
982
983
984
985
986
987
988
989
990
991
992
993
994
995
996
997
998
999
1000
1001
1002
1003
1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1022
1023
1024
1025
1026
1027
1028
1029
1030
1031
1032
1033
1034
1035
1036
1037
1038
1039
1040
1041
1042
1043
1044
1045
1046
1047
1048
1049
1050
1051
1052
1053
1054
1055
1056
1057
1058
1059
1060
1061
1062
1063
1064
1065
1066
1067
1068

1194
1195
1196
1197
1198
1199
1200
1201
1202
1203
1204
1205
1206
1207
1208
1209
1210
1211
1212
1213
1214
1215
1216
1217
1218
1219
1220
1221
1222
1223
1224
1225
1226
1227
1228
1229
1230
1231
1232
1233
1234
1235
1236
1237
1238
1239
1240
1241
1242
1243
1244
1245
1246
1247
1248
1249
1250
1251
1252
1253
1254
1255
1256
1257
1258
1259
1260
1261
1262
1263
1264
1265
1266
1267
1268
1269
1270
1271
1272
1273
1274
1275
1276
1277
1278
1279
1280
1281
1282
1283
1284
1285
1286
1287
1288
1289
1290
1291
1292
1293
1294
1295
1296
1297
1298
1299
1300
1301
1302
1303
1304
1305
1306
1307
1308
1309
1310
1311
1312
1313
1314
1315
1316
1317
1318
1319
1320
1321
1322
1323
1324
1325
1326
1327
1328
1329
1330
1331
1332
1333
1334
1335
1336
1337
1338
1339
1340
1341
1342
1343
1344
1345
1346
1347
1348
1349
1350
1351
1352
1353
1354
1355
1356
1357
1358
1359
1360
1361
1362
1363
1364
1365
1366
1367
1368
1369
1370
1371
1372
1373
1374
1375
1376
1377
1378
1379
1380
1381
1382
1383
1384
1385
1386
1387
1388
1389
1390
1391
1392
1393


In [ ]:
for i in range(len(loss_acc_list)):
    print('Iteration:',i)
    print('loss=',loss_acc_list[i][0])
    print('acc=',loss_acc_list[i][1])
    print('time=',loss_acc_list[i][2])

# Get latent factor and Each weight

In [54]:
U, Y, A, A1, Au, Ay, Aa, Av,E,B =sess.run([user_latent, item_latent, aux_item, W1, Wu, Wy, Wa, Wv,embedding,Beta])

In [55]:
print('User latent shape: ',U.shape)
print('photo latent shape: ', Y.shape)
print('Auxilary latent shape: ',A.shape)
print('W1 weight shape: ',A1.shape)
print('Wu weight shape:',Au.shape)
print('Wy weight shape:', Ay.shape)
print('Wa weight shape:',Aa.shape)
print('Wv weight shape:',Av.shape)
print('Beta shape:',B.shape)

User latent shape:  (1489, 100)
photo latent shape:  (88, 100)
Auxilary latent shape:  (88, 100)
W1 weight shape:  (1489, 100)
Wu weight shape: (100, 100)
Wy weight shape: (88, 100, 100)
Wa weight shape: (100, 100)
Wv weight shape: (100, 3374)
Beta shape: (1489, 150)


# Testing Part

In [ ]:
# without embedding
result=np.zeros((test_amount,88))
RS=np.zeros((test_amount,88))
#test_idx --> Test 的 index

test_yes_id=[]
for s in range(test_amount):
    print(s,test_idx[s])

    yes=[]
    sample=random.sample(train_t[test_idx[s]],len(train_t[test_idx[s]])) #從training part 的positive feedback 取出YouTuber 當成Auxilary
    #sample=result_yes_id[now]
    test_yes_id.append(sample)
    alpha=np.zeros([len(sample)])
    
    for a in range(len(sample)):
        r =np.max(YouTuber_category[sample[a]]*user_category_norm[test_idx[s]]) #sample a 的category vec *user_category vec
        #print(test_idx[s])
        #print(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0)))
        alpha[a]=np.dot(A1[test_idx[s]],(relu(np.dot(Au,np.expand_dims(U[test_idx[s]],0).T)+np.dot(Ay[sample[a]],np.expand_dims(Y[sample[a]],0).T)+np.dot(Aa,
                            np.expand_dims(A[sample[a]],0).T)+ np.dot(Av,np.expand_dims(all_3374[sample[a]],0).T))))*r
    mul=np.zeros((1,128))
    print('alpha------------',alpha)
    print('softmax alpha--------------',softmax(alpha))
    for i in range(len(sample)):
        mul+=softmax(alpha)[i]*A[sample[i]] #attention alpha*Ai part 
    new_mul=mul+U[test_idx[s]]  #(U+auxilary)
    for k in range(88):
        result[s][k]=np.dot(new_mul,Y[k].T) #(U+auxilary)*photo latent factor
        RS[s][k] = np.dot(new_mul,Y[k].T)+np.dot(B[test_idx[s]], all_3374[k].T)
#print(RS[s])

In [57]:
#with Embedding
result=np.zeros((test_amount,88))
RS=np.zeros((test_amount,88))
#test_idx --> Test 的 index

test_yes_id=[]
for s in range(test_amount):
    print(s,test_idx[s])

    yes=[]
    sample=random.sample(train_t[test_idx[s]],len(train_t[test_idx[s]])) #從training part 的positive feedback 取出YouTuber 當成Auxilary
    #sample=result_yes_id[now]
    test_yes_id.append(sample)
    alpha=np.zeros([len(sample)])
    
    for a in range(len(sample)):
        r =np.max(YouTuber_category[sample[a]]*user_category_norm[test_idx[s]]) #sample a 的category vec *user_category vec
        #print(test_idx[s])
        #print(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0)))
        alpha[a]=np.dot(A1[test_idx[s]],(relu(np.dot(Au,np.expand_dims(U[test_idx[s]],0).T)+np.dot(Ay[sample[a]],np.expand_dims(Y[sample[a]],0).T)+np.dot(Aa,
                            np.expand_dims(A[sample[a]],0).T)+ np.dot(Av,np.expand_dims(all_3374[sample[a]],0).T))))*r
    mul=np.zeros((1,100))
    print('alpha------------',alpha)
    print('softmax alpha--------------',softmax(alpha))
    for i in range(len(sample)):
        mul+=softmax(alpha)[i]*A[sample[i]] #attention alpha*Ai part 
    new_mul=mul+U[test_idx[s]]  #(U+auxilary)
    for k in range(88):
        result[s][k]=np.dot(new_mul,Y[k].T) #(U+auxilary)*photo latent factor
        RS[s][k] = np.dot(new_mul,Y[k].T)+np.dot(B[test_idx[s]], np.dot(E,all_3374[k].T))
#print(RS[s])

0 1340
alpha------------ [-9.03361733e-17 -9.63105776e-17  1.53212032e-40  3.73811776e-17
 -3.86909646e-16  6.36569976e-18]
softmax alpha-------------- [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
1 826
alpha------------ [5.65538460e-15 1.69332221e-38 0.00000000e+00 1.04199654e-15
 5.99174227e-15]
softmax alpha-------------- [0.2 0.2 0.2 0.2 0.2]
2 262
alpha------------ [-4.47867921e-16  2.01351913e-41 -1.31513464e-16  3.81900273e-40
 -5.14546730e-17  4.52562736e-16]
softmax alpha-------------- [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
3 682
alpha------------ [-2.35402656e-20 -2.45445355e-44 -1.23400221e-20 -2.10173188e-20
  1.91155778e-20 -8.65726628e-21 -1.00102498e-20  2.52342508e-20]
softmax alpha-------------- [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
4 463
alpha------------ [ 8.89523264e-19  1.81569297e-18 -4.44921570e-43  1.38206529e-42
  8.81612539e-19  2.23625013e-43 -4.05501891e-18]
softmax alpha-------------- [0.142857

38 421
alpha------------ [1.93438188e-25 3.41143458e-25 4.01574293e-24 6.21714049e-25
 2.15430448e-24 3.73711307e-25 2.98851202e-25 4.61687582e-25
 6.36441078e-24 6.82121401e-25]
softmax alpha-------------- [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
39 840
alpha------------ [ 3.69466193e-40  5.11035328e-18 -8.57027428e-17 -8.94320131e-18
 -4.98360297e-17  1.04236422e-16]
softmax alpha-------------- [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
40 1293
alpha------------ [ 5.06284142e-18  6.98649809e-42  2.04331068e-17  7.98978063e-19
 -1.11812520e-19  7.64857279e-19  1.10382773e-18]
softmax alpha-------------- [0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
41 469
alpha------------ [ 6.43808234e-19  1.10527727e-18  1.03755407e-18 -6.32656247e-19
  3.90919201e-18 -5.66751109e-19  2.34396069e-19]
softmax alpha-------------- [0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
42 879
alpha------------ [-6.04273

76 516
alpha------------ [ 9.06864858e-28  1.17988668e-50  1.27655840e-50  3.92231793e-28
 -6.52601082e-27  9.19951334e-51  6.05032889e-27  2.01115243e-27
 -2.31179810e-27  2.60038383e-28 -9.21494165e-28]
softmax alpha-------------- [0.09090909 0.09090909 0.09090909 0.09090909 0.09090909 0.09090909
 0.09090909 0.09090909 0.09090909 0.09090909 0.09090909]
77 719
alpha------------ [-4.40111927e-39 -5.48130974e-15  4.25643336e-16 -2.03296916e-15
 -3.87819421e-39]
softmax alpha-------------- [0.2 0.2 0.2 0.2 0.2]
78 541
alpha------------ [-2.90078819e-18 -2.81245084e-18 -1.74166167e-16 -6.83406389e-16
 -1.39509310e-17 -1.61079198e-15]
softmax alpha-------------- [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
79 97
alpha------------ [ 1.07475104e-16  2.66153625e-16 -1.16867947e-17  7.99173220e-41
 -2.59120290e-16 -1.34233045e-16]
softmax alpha-------------- [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
80 458
alpha------------ [-1.63429324e-18  7.7

112 764
alpha------------ [ 2.52531509e-17 -3.80439547e-16  2.05036890e-17  1.36403977e-16
 -5.32467840e-17  3.91247578e-40]
softmax alpha-------------- [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
113 61
alpha------------ [ 1.19287274e-40  2.75295515e-24  4.16454092e-24  6.15587561e-25
  2.24381330e-25  1.62486734e-48  2.85882185e-24  1.34429386e-24
  5.01548586e-25 -9.88409256e-26]
softmax alpha-------------- [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
114 1345
alpha------------ [ 1.36636364e-18  1.14725155e-42 -1.52808262e-18  8.54773898e-18
 -1.60344645e-18 -3.80971187e-19  7.40400222e-19]
softmax alpha-------------- [0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
115 1152
alpha------------ [-3.63733166e-15  8.74926317e-16 -3.16016484e-15 -1.01366485e-39
 -1.30587408e-29]
softmax alpha-------------- [0.2 0.2 0.2 0.2 0.2]
116 1290
alpha------------ [-1.82547087e-40 -2.00784416e-41 -6.50722161e-17 -8.43388151e-17
 -2.90726611e-16 -

148 427
alpha------------ [ 3.31165993e-19  4.21524545e-43  7.58946810e-19  2.08270684e-18
 -1.16996459e-18 -2.33370438e-18  4.79006229e-18]
softmax alpha-------------- [0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
149 158
alpha------------ [-7.97584733e-41  4.69359489e-16  1.13240073e-16 -3.75784102e-16
  1.47480380e-16  1.04041546e-16]
softmax alpha-------------- [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]


In [58]:
#取出test的資料
testRS = np.zeros((test_amount,yt_test_amount)) #shape 150*20
target = np.zeros((test_amount,yt_test_amount))
#test_t 是true的
#test_f 是false的
        
for z in range(test_amount):
    user_id = test_idx[z]
    #positive target YouTuber list
    youtube_t = test_t[z] 
    #not target YouTuber list
    youtube_f = test_f[z]
    
    #前兩個放target的RS
    for i in range(len(youtube_t)):
        testRS[z][i] = RS[z][youtube_t[i]]
        target[z][i] = 1
    for i in range(len(youtube_f)):
        testRS[z][i+len(youtube_t)] = RS[z][youtube_f[i]]

In [70]:
target.shape

(150, 20)

In [71]:
testRS.shape

(150, 20)

In [59]:
sumtarget = 0
for i in range(len(target)):
    #print(np.sum(target[i]))
    sumtarget += np.sum(target[i])
print('num of positive data in testing:',sumtarget)
print('total testing data:',test_amount*yt_test_amount)

num of positive data in testing: 966.0
total testing data: 3000


In [60]:
def topN(sortlist,n):
    topList = []
    for i in range(n):
        topList.append(sortlist.index(max(sortlist)))
        #print(max(sortlist))
        #print(sortlist.index(max(sortlist)))
        sortlist[sortlist.index(max(sortlist))] = -1000000000
    return topList

In [182]:
count_0_all = []
for i in range(len(testRS)):
    top_0 = topN(list(testRS[i]),int(np.sum(target[i])))
    count_0_all.append(top_0)
    #print(top_0)

acc_0 = 0
total = 0
for i in range(len(count_0_all)):
    for j in range(len(count_0_all[i])):
        #print(int(np.sum(target[i])))
        total+=int(np.sum(target[i]))
        if count_0_all[i][j] < int(np.sum(target[i])): #代表是0或1 (也就是target)
            acc_0 += 1
avg_acc = acc_0/100
#print('avg_accuarcy for count_0:',avg_acc)
#print(acc_0)

In [ ]:
acc_0

In [ ]:
total

# Top 1 

In [62]:
def F1_score(prec,rec):
    f1 = (2*prec*rec)/(prec+rec)
    return f1

In [179]:
correct = 0
for i in range(len(testRS)):
    top_0 = topN(list(testRS[i]),1) #取一個
    count_0_all.append(top_0)
    #print(np.sum(target[i]))
    #print(top_0)
    if top_0[0] < int(np.sum(target[i])):
        correct += 1

In [74]:
top1_prec = correct/len(testRS)
top1_recall = correct/(sumtarget)
print('prec ',top1_prec,'recall ',top1_recall)

prec  0.6466666666666666 recall  0.10041407867494824


In [75]:
#f1 score
print('F1_score:',F1_score(top1_prec,top1_recall))

F1_score: 0.17383512544802868


# Top 3

In [76]:
correct = 0
for i in range(len(testRS)):
    top_3 = topN(list(testRS[i]),3) #取一個
    count_0_all.append(top_3)
    #print(top_3)
    for j in range(len(top_3)):
        if top_3[j] < int(np.sum(target[i])):
            correct += 1

In [77]:
top3_prec = correct/(len(testRS)*3)
top3_recall = correct/(sumtarget)
print('prec ',top3_prec,'recall ',top3_recall)

prec  0.5733333333333334 recall  0.2670807453416149


In [78]:
#f1 score
print('F1_score:',F1_score(top3_prec,top3_recall))

F1_score: 0.3644067796610169


# Top 5

In [180]:
correct = 0
for i in range(len(testRS)):
    top_5 = topN(list(testRS[i]),5) #取一個
    count_0_all.append(top_5)
    #print(top_5)
    for j in range(len(top_5)):
        if top_5[j] < int(np.sum(target[i])):
            correct += 1

In [80]:
top5_prec = correct/(len(testRS)*5)
top5_recall = correct/(sumtarget)
print('prec ',top5_prec,'recall ',top5_recall)

prec  0.5626666666666666 recall  0.43685300207039335


In [81]:
#f1 score
print('F1_score:',F1_score(top5_prec,top5_recall))

F1_score: 0.4918414918414918


In [181]:
# pre_list
"""
test_amount = 150
yt_test_amount = 20
"""
all_sort = []
pre_matrix = np.zeros(shape=(test_amount,yt_test_amount))
for i in range(test_amount):
    top_5 = topN(list(testRS[i]),5) #取一個
    #print(top_5)
    all_sort.append(topN(list(testRS[i]),len(testRS[i])))
    for j in range(len(top_5)):
        pre_matrix[i][top_5[j]] = 1

In [129]:
pre_matrix.shape

(150, 20)

In [130]:
target.shape

(150, 20)

# NDCG

In [164]:
"""
NDCG
https://daiwk.github.io/posts/nlp-ndcg.html
"""
#Ideal DCG，理想状况下的DCG。也就是说，相关性完全由高到低排序时算出的DCG：
def IDCG(ideal_list): #ideal_list example = [1,1,1,1,1,0,0,....]
    idcg=0
    for i in range(len(ideal_list)):
        #print((2**true_list[i]-1),math.log2(i+2))
        idcg+= (2**ideal_list[i]-1)/ math.log2(i+2)
    #print('idcg',idcg)
    return idcg
def DCG(prec_list): #找出前n名的[1,1,1,0,...]
    dcg=0
    for i in range(len(prec_list)):
        dcg+= (2**prec_list[i]-1)/ math.log2(i+2)
    #print('dcg',dcg)
    return dcg

In [177]:
total_ndcg = 0
num_ndcg = 5
for m in range(test_amount):
    idcg = IDCG([1]*num_ndcg)
    pre_list = []
    for s in all_sort[m][:num_ndcg]:
        #print(s)
        #print(target[m][s])
        pre_list.append(target[m][s])
    dcg = DCG(pre_list)
    ndcg = dcg/idcg
    #print(ndcg)
    total_ndcg += ndcg
avg_ndcg = total_ndcg/test_amount
print('NDCG:',avg_ndcg)

NDCG: 0.5775862147041461


# MAP

In [174]:
from sklearn.metrics import average_precision_score

In [175]:
"""
>>> y_true = np.array([0, 0, 1, 1])
>>> y_scores = np.array([0.1, 0.4, 0.35, 0.8])
>>> average_precision_score(y_true, y_scores)
"""
total_prec = 0
for u in range(test_amount):
    y_true = target[u]
    y_scores = pre_matrix[u]
    total_prec+=average_precision_score(y_true, y_scores)
MAP = total_prec/test_amount

In [178]:
print('MAP',MAP)

MAP 0.47533774439068566
